<a href="https://colab.research.google.com/github/expeditive/machine-learning/blob/main/project/deploying%20as%20public%20api/deploying_ML_model_as_public_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pickle5
  Running setup.py clean for pickle5
Failed to build pickle5
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pickle5)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/

In [2]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [3]:
app = FastAPI()

In [4]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [5]:
class model_input(BaseModel):
  Pregnancies : int
  Glucose : int
  BloodPressure : int
  SkinThickness : int
  Insulin : int
  BMI : float
  DiabetesPedigreeFunction : float
  Age : int


In [7]:
#loading the saved model
diabetes_model = pickle.load(open('/content/diabetes_model.sav','rb'))

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.0.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
@app.post('/diabetes_prediction')
def diabetes_pred(input_parameters : model_input):

  input_data = input_parameters.json()
  input_dictionary = json.loads(input_data)

  preg = input_dictionary['Pregnancies']
  glu = input_dictionary['Glucose']
  bp = input_dictionary['BloodPressure']
  skin = input_dictionary['SkinThickness']
  insulin = input_dictionary['Insulin']
  bmi = input_dictionary['BMI']
  dpf = input_dictionary['DiabetesPedigreeFunction']
  age = input_dictionary['Age']


  input_list = [preg, glu, bp, skin, insulin, bmi, dpf, age]

  prediction = diabetes_model.predict([input_list])


  if prediction[0] == 0:
    return 'The person is not Diabetic'

  else:
    return 'The Person is Diabetic'


In [ ]:
ngrok.set_auth_token("2xMT1k0ZM0lesnJj4nioL4Xf8Ka_7kfp1htwryT3qpxseEUo5")
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:',ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app,port=8000)

INFO:     Started server process [226]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://bf60-34-57-180-212.ngrok-free.app
INFO:     2409:4089:a284:427:9153:3e68:8dd9:8a13:0 - "POST /diabetes_prediction HTTP/1.1" 200 OK


<ipython-input-8-37bec6134029>:4: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  input_data = input_parameters.json()
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
